---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

## **EnumOutputParser**

* 언어 모델의 출력을 `미리 정의된 열거형`(`Enum`) 값 중 하나로 `파싱`하는 도구

### **주요 특징**

* **열거형 파싱**: `문자열 출력`을 `미리 정의된 Enum 값`으로 변환
* **타입 안전성**: `파싱된 결과`가 `반드시 정의된 Enum 값` 중 하나임을 보장
* **유연성**: `공백`이나 `줄바꿈` 문자를 `자동`으로 `처리`

### **사용 방법**

* `EnumOutputParser` 언어 모델의 출력에서 `유효한 Enum 값`을 `추출`하는 데 유용
* 출력 데이터의 `일관성을 유지`하고 `예측 가능성`을 높일 수 있음

<br>

* 파서 사용: `미리 정의된 Enum 값을 설정` → 해당 값을 기준으로 `문자열 출력`을 파싱

In [1]:
# 1_새 프롬프트 생성하기
from langsmith import Client
from langchain.prompts import PromptTemplate                            # Langchain에서 프롬프트 템플릿을 만들기 위한 모듈 임포트
from langchain.prompts import ChatPromptTemplate
from langsmith import Client

import os
import json


# 클라이언트 생성 
api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=api_key)

In [2]:
from langchain.output_parsers.enum import EnumOutputParser               # EnumOutputParser 출력을 위한 임포트

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langsmith import traceable                                     # LangSmith 추적 설정

# LLM 초기화
gemini_lc = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0.7,                                    
        max_output_tokens=4096,
    )

---

* `enum` 모듈 사용 → `Colors` 클래스 정의하기
  
* `Colors`클래스 = `Enum` 상속 → `RED` `GREEN` `BLUE`의 세 가지 색상 값을 가짐

In [ ]:
from enum import Enum

class Colors(Enum):
    RED = "빨간색"
    GREEN = "초록색"
    BLUE = "파란색"

# EnumOutputParser 인스턴스 생성
parser = EnumOutputParser(enum=Colors)
print(parser)                                   # enum=<enum 'Colors'>
print(type(parser))                             # <class 'langchain.output_parsers.enum.EnumOutputParser'>

* `프롬프트` = **사람의 정보`({person})`** + **파싱 지침`({instructions})`** 포함
  
* `parser.get_format_instructions() 함수`를 호출하여 파싱 지침 가져오기
  
* `체인` = **프롬프트** | **`LLM`모델** | **파서**

In [ ]:
# 프롬프트 템플릿 생성하기
prompt = PromptTemplate.from_template(
    """다음의 물체는 어떤 색깔인가요?

Object: {object}

Instructions: {instructions}"""
    # 파서에서 지시사항 형식을 가져와 부분적으로 적용하기
).partial(instructions=parser.get_format_instructions())


print(prompt)                           
# input_variables=['object'] input_types={} partial_variables={'instructions': 'Select one of the following options: 빨간색, 초록색, 파란색'} template='다음의 물체는 어떤 색깔인가요?\n\nObject: {object}\n\nInstructions: {instructions}'
print(type(prompt))                  # <class 'langchain_core.prompts.prompt.PromptTemplate'>

# 체인 생성 = 프롬프트 + LLM 모델 + 파서
chain = prompt | gemini_lc | parser

* `chain.invoke` 함수를 사용하여 **하늘** 에 대한 정보를 요청

In [ ]:
response = chain.invoke({"object": "하늘"})                 # "하늘" 에 대한 체인 호출 실행
print(response)

<small>

* 셀 출력 (1.3s)

    ```plaintext
    Colors.BLUE
    ```

---

### 실험

* **Q.** 클래스에서 벗어난 `object`를 물어보면 어떻게 될까?

In [ ]:
response2 = chain.invoke({"object": "나무"})                # "나무" 에 대한 체인 호출 실행
print(response2)

<small>    

* 셀 출력 (1.0s) → **파싱 실패!**
    
    ```python
    ---------------------------------------------------------------------------
    ValueError                                Traceback (most recent call last)
    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain/output_parsers/enum.py:28, in EnumOutputParser.parse(self, response)
        27 try:
    ---> 28     return self.enum(response.strip())
        29 except ValueError as e:

    File ~/.pyenv/versions/3.13.5/lib/python3.13/enum.py:726, in EnumType.__call__(cls, value, names, module, qualname, type, start, boundary, *values)
        725         value = (value, names) + values
    --> 726     return cls.__new__(cls, value)
        727 # otherwise, functional API: we're creating a new Enum type

    File ~/.pyenv/versions/3.13.5/lib/python3.13/enum.py:1203, in Enum.__new__(cls, value)
    1202 if result is None and exc is None:
    -> 1203     raise ve_exc
    1204 elif exc is None:

    ValueError: '나무' is not a valid Colors

    The above exception was the direct cause of the following exception:

    OutputParserException                     Traceback (most recent call last)
    Cell In[7], line 1
    ----> 1 response2 = chain.invoke({"object": "나무"})                # "나무" 에 대한 체인 호출 실행
        2 print(response2)

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/runnables/base.py:3046, in RunnableSequence.invoke(self, input, config, **kwargs)
    3044                 input_ = context.run(step.invoke, input_, config, **kwargs)
    3045             else:
    -> 3046                 input_ = context.run(step.invoke, input_, config)
    3047 # finish the root run
    3048 except BaseException as e:

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/output_parsers/base.py:196, in BaseOutputParser.invoke(self, input, config, **kwargs)
        188 @override
        189 def invoke(
        190     self,
    (...)    193     **kwargs: Any,
        194 ) -> T:
        195     if isinstance(input, BaseMessage):
    --> 196         return self._call_with_config(
        197             lambda inner_input: self.parse_result(
        198                 [ChatGeneration(message=inner_input)]
        199             ),
        200             input,
        201             config,
        202             run_type="parser",
        203         )
        204     return self._call_with_config(
        205         lambda inner_input: self.parse_result([Generation(text=inner_input)]),
        206         input,
        207         config,
        208         run_type="parser",
        209     )

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/runnables/base.py:1939, in Runnable._call_with_config(self, func, input_, config, run_type, serialized, **kwargs)
    1935     child_config = patch_config(config, callbacks=run_manager.get_child())
    1936     with set_config_context(child_config) as context:
    1937         output = cast(
    1938             "Output",
    -> 1939             context.run(
    1940                 call_func_with_variable_args,  # type: ignore[arg-type]
    1941                 func,
    1942                 input_,
    1943                 config,
    1944                 run_manager,
    1945                 **kwargs,
    1946             ),
    1947         )
    1948 except BaseException as e:
    1949     run_manager.on_chain_error(e)

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/runnables/config.py:429, in call_func_with_variable_args(func, input, config, run_manager, **kwargs)
        427 if run_manager is not None and accepts_run_manager(func):
        428     kwargs["run_manager"] = run_manager
    --> 429 return func(input, **kwargs)

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/output_parsers/base.py:197, in BaseOutputParser.invoke.<locals>.<lambda>(inner_input)
        188 @override
        189 def invoke(
        190     self,
    (...)    193     **kwargs: Any,
        194 ) -> T:
        195     if isinstance(input, BaseMessage):
        196         return self._call_with_config(
    --> 197             lambda inner_input: self.parse_result(
        198                 [ChatGeneration(message=inner_input)]
        199             ),
        200             input,
        201             config,
        202             run_type="parser",
        203         )
        204     return self._call_with_config(
        205         lambda inner_input: self.parse_result([Generation(text=inner_input)]),
        206         input,
        207         config,
        208         run_type="parser",
        209     )

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/output_parsers/base.py:250, in BaseOutputParser.parse_result(self, result, partial)
        234 @override
        235 def parse_result(self, result: list[Generation], *, partial: bool = False) -> T:
        236     """Parse a list of candidate model Generations into a specific format.
        237 
        238     The return value is parsed from only the first Generation in the result, which
    (...)    248         Structured output.
        249     """
    --> 250     return self.parse(result[0].text)

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain/output_parsers/enum.py:34, in EnumOutputParser.parse(self, response)
        29 except ValueError as e:
        30     msg = (
        31         f"Response '{response}' is not one of the "
        32         f"expected values: {self._valid_values}"
        33     )
    ---> 34     raise OutputParserException(msg) from e

    OutputParserException: Response '나무' is not one of the expected values: ['빨간색', '초록색', '파란색']
    For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE
    ```

<small>

* 파싱 실패 원인 분석
  * `EnumOutputParser`를 사용해서 모델의 응답을 `미리 정해둔 값(Enum)` 중 하나로 `파싱`하기로 정의해둠
  * `Colors` 클래스 = `RED`, `GREEN`, `BLUE`

<br>

* ![문제, 원인, 해결](../data/enumoutputparser_error.png)

<br>

* 핵심 오류 코드 부분

    ```python
    OutputParserException: Response '나무' is not one of the expected values: ['빨간색', '초록색', '파란색']
    ```

  * `나무` 입력 = `EnumOutputParser`가 예상하는 값 리스트에 없음 = **에러 발생** = **파싱 실패**

---

* ![해결방법 시도 방법 3가지](../data/enumoutputparser_try.png)

---

#### 해결 방법 1 - **프롬프트 수정** 하기

* `LLM`이 `올바른 Enum 값`만 `출력`하게 `유도`하는 것

* 장점:

  * **`Enum 그대로 두고도`** 다양한 입력을 처리할 수 있음
  * 파싱 오류를 줄이는 가장 **가벼운** 방법

<small>

* 프롬프트 수정 예시 → 아래와 같이 수정 후 `LLM` 모델이 **나무** 같은 입력에 **`GREEN`** 같은 적절한 답을 하도록 `유도` 가능

    ```python

    prompt = PromptTemplate.from_template(
        """다음의 물체에 어울리는 색깔을 아래 목록 중에서 골라주세요:

    객체: {object}

    가능한 색상: 빨간색, 초록색, 파란색

    반드시 가능한 색상 중 하나로만 답해주세요.

    Instructions: {instructions}"""
    ).partial(instructions=parser.get_format_instructions())

    ```

In [8]:
# 해결방법1 = 프롬프트를 수정해보기

prompt2 = PromptTemplate.from_template(
    """다음의 물체에 어울리는 색깔을 아래 목록 중에서 골라주세요:

객체: {object}

가능한 색상: 빨간색, 초록색, 파란색

반드시 가능한 색상 중 하나로만 답해주세요.

Instructions: {instructions}"""
).partial(instructions=parser.get_format_instructions())

In [ ]:
# 체인 생성 = 프롬프트 + LLM 모델 + 파서
chain = prompt2 | gemini_lc | parser

# 나무에 대해서 물어보기
response2 = chain.invoke({"object": "나무"})
print(response2)                                        # Colors.GREEN (1.0s)

#### 해결 방법 2 - **Enum 클래스 수정 (색상 확장)**

* `Enum`에 `더 많은 색상 값`을 `추가`해서 `유연성`을 `높이는 것`

* 장점

  * 더 `다양한 입력` (예: `나무`, `흙`, `노을`)에 대해 `잘 대응` 가능

  * 파싱 `실패`가 `줄어듦`

* 단점

  * `Enum`이 **`너무 커지면 관리가 어려워짐`**

  * `너무 유연`하면, **분류 기준이 흐려질 수 있음**

In [10]:
class Colors2(Enum):
    RED = "빨간색"
    GREEN = "초록색"
    BLUE = "파란색"
    BROWN = "갈색"
    YELLOW = "노란색"
    BLACK = "검은색"

# EnumOutputParser 인스턴스 생성
parser2 = EnumOutputParser(enum=Colors2)

In [12]:
prompt2 = PromptTemplate.from_template(
    """다음의 물체에 어울리는 색깔을 아래 목록 중에서 골라주세요:

객체: {object}

가능한 색상: 빨간색, 초록색, 파란색

반드시 가능한 색상 중 하나로만 답해주세요.

Instructions: {instructions}"""
).partial(instructions=parser.get_format_instructions())

In [ ]:
# 체인 생성 = 프롬프트 + LLM 모델 + 파서
chain2 = prompt2 | gemini_lc | parser2

# 밤하늘 
response3 = chain.invoke({"object": "밤하늘"})
print(response3)                                    # Colors.BLUE (0.6s)

#### 해결 방법 3 - **Parser Customizing**

* 기존 EnumOutputParser로는 불가능한 `다중 Enum 값 출력`을 처리하기 위해, `파서를 직접 정의`해 `원하는 형식으로 파싱`하는 방식
  * 예시: `List` ...

* 장점

  * `여러 개의 Enum 값을 동시에` 받을 수 있어 `유연한 답변 처리`가 가능

  * `출력 형`식을 `더 세밀하게 제`어하고, `오류 처리`도 직접 `커스터마이징 가능`

* 단점

  * 기본 parser보다 `구현 복잡도가 높고`, `유지 관리`가 필요

  * 형식이 예상과 다를 경우 `직접 예외 처리를 추가`해야 함 
    * 예시: 쉼표 누락, 오탈자...

In [20]:
from typing import List, Type
from enum import Enum
from pydantic import Field
from langchain.schema import BaseOutputParser, OutputParserException
import difflib

# 예외 처리 포함한 커스텀 Enum 리스트 파서 클래스
class EnumListOutputParser(BaseOutputParser):
    # Enum 클래스 타입 지정 (중요!)
    enum_class: Type[Enum] = Field(...)
    min_choices: int = Field(default=2)
    max_choices: int = Field(default=3)

    def parse(self, text: str) -> List[Enum]:
        # 쉼표 기준으로 나누고, 앞뒤 공백 제거 후 빈 문자열 제거
        items = [item.strip() for item in text.split(",") if item.strip()]

        # 최소/최대 선택 개수 검사
        if not (self.min_choices <= len(items) <= self.max_choices):
            raise OutputParserException(
                f"색상은 최소 {self.min_choices}개 이상, 최대 {self.max_choices}개 이하로 선택해야 합니다. 현재 선택 수: {len(items)}"
            )

        result = []
        enum_values = [e.value for e in self.enum_class]

        for item in items:
            if item in enum_values:
                enum_value = next(e for e in self.enum_class if e.value == item)
                result.append(enum_value)
            else:
                # 오타 체크: 유사한 값 찾아서 사용
                closest = difflib.get_close_matches(item, enum_values, n=1, cutoff=0.6)
                if closest:
                    enum_value = next(e for e in self.enum_class if e.value == closest[0])
                    result.append(enum_value)
                else:
                    raise OutputParserException(
                        f"잘못된 값입니다: '{item}' (허용된 값: {', '.join(enum_values)})"
                    )
        return result

    def get_format_instructions(self) -> str:
        values = [e.value for e in self.enum_class]
        return (
            f"다음 중 {self.min_choices}~{self.max_choices}개의 색상을 쉼표로 구분하여 출력하세요: "
            + ", ".join(values)
        )

# 확장된 색상 Enum 클래스
class Colors3(Enum):
    RED = "빨간색"
    GREEN = "초록색"
    BLUE = "파란색"
    BROWN = "갈색"
    YELLOW = "노란색"
    BLACK = "검은색"
    WHITE = "하얀색"
    ORANGE = "주황색"
    PURPLE = "보라색"
    PINK = "분홍색"
    GRAY = "회색"

# 파서 인스턴스 생성 (최소 2개, 최대 3개 선택)
parser3 = EnumListOutputParser(enum_class=Colors3, min_choices=2, max_choices=3)

In [ ]:
# 테스트 해보기

test_input = "빨간색, 파란색, 분홍색"
try:
    parsed_colors = parser3.parse(test_input)
    print("파싱 결과:", [color.name for color in parsed_colors])  # Enum 이름 출력
except OutputParserException as e:
    print("파싱 에러:", e)                                        # 파싱 결과: ['RED', 'BLUE', 'PINK'] 

In [22]:
# 프롬프트 커스터마이징

# from langchain import PromptTemplate                          # 사전에 임포트했으므로 여기에서는 생략

# Color3의 모든 색상 값을 쉼표로 구분해서 나열하는 부분
colors_list = ", ".join([color.value for color in Colors3])     

# 프롬프트_3
prompt3 = PromptTemplate.from_template(
    f"""다음의 물체에 어울리는 색깔을 아래 목록 중에서 2~3가지 골라주세요.
쉼표로 구분해서 답변해주세요.

객체: {{object}}

가능한 색상: {colors_list}

Instructions: {{instructions}}"""
).partial(instructions=parser3.get_format_instructions())

In [24]:
# 체인 생성 = 프롬프트 + LLM 모델 + 파서
chain3 = prompt3 | gemini_lc | parser3

# 나무에 대해서 물어보기
response2 = chain3.invoke({"object": "나무"})
print(response2)                                        
# [<Colors3.BROWN: '갈색'>, <Colors3.GREEN: '초록색'>, <Colors3.YELLOW: '노란색'>] (0.7s)

[<Colors3.BROWN: '갈색'>, <Colors3.GREEN: '초록색'>, <Colors3.YELLOW: '노란색'>]


In [ ]:
response3 = chain3.invoke({"object":"밤하늘"})
print(response3)
# [<Colors3.BLUE: '파란색'>, <Colors3.BLACK: '검은색'>, <Colors3.PURPLE: '보라색'>] (0.7s)

In [ ]:
response3 = chain3.invoke({"object":"밤하늘"})
print(response3)
# [<Colors3.BLUE: '파란색'>, <Colors3.BLACK: '검은색'>, <Colors3.PURPLE: '보라색'>] (0.7s)

In [ ]:
# 여러 다른 객체에 대해 테스트 해보기

test_objects = ["햇볕", "바다", "산", "사막", "꽃밭", "우주", "숲속", "저녁 하늘"]

for obj in test_objects:
    try:
        response = chain3.invoke({"object": obj})
        print(f"{obj}에 어울리는 색상:", [color.name for color in response])
    except Exception as e:
        print(f"{obj} 처리 중 에러 발생:", e)

<small>

* 셀 출력 (1m 7.7s)
  * 햇볕 ~ 숲속: 22s
  * 저녁 하늘 유추에 시간이 많이 소요됨 (22s ~ 1m 7.7s) → 색상 후보가 다양해서 적합한 색상을 추론하는 데 시간이 오래 걸렸을 것으로 생각됨

    ```markdown
    햇볕에 어울리는 색상: ['YELLOW', 'ORANGE', 'WHITE']
    바다에 어울리는 색상: ['BLUE', 'GREEN', 'GRAY']
    산에 어울리는 색상: ['GREEN', 'BROWN', 'GRAY']
    사막에 어울리는 색상: ['BROWN', 'YELLOW', 'ORANGE']
    꽃밭에 어울리는 색상: ['RED', 'GREEN', 'YELLOW']
    우주에 어울리는 색상: ['BLACK', 'BLUE', 'PURPLE']
    숲속에 어울리는 색상: ['GREEN', 'BROWN', 'YELLOW']
    저녁 하늘에 어울리는 색상: ['PURPLE', 'ORANGE', 'BLUE']
    ```

In [ ]:
# 의미 없는 단어나 색상과 관련 없는 객체로 예외 테스트
test_objects_with_invalid_values = [
    "내 마음속",            # 관련 없는 단어
    "푸른 바다",            # 색상과 관련 없는 단어
    "가방",                # 물체와 색상이 관계 없는 예시
    "깊은 우주",            # 이상한 표현
    "~~~",                # 특수문자
    "12345"               # 숫자
]

for obj in test_objects_with_invalid_values:
    try:
        response = chain3.invoke({"object": obj})
        print(f"{obj}에 어울리는 색상:", [color.name for color in response])
    except OutputParserException as e:
        print(f"에러 발생! {obj}: {e}")
    except Exception as e:
        print(f"알 수 없는 에러 발생! {obj}: {e}")

<small>

* 셀 출력 (5.2s)

    ```markdown
    내 마음속에 어울리는 색상: ['BLUE', 'PURPLE', 'PINK']
    푸른 바다에 어울리는 색상: ['BLUE', 'GREEN', 'GRAY']
    가방에 어울리는 색상: ['BLACK', 'BROWN', 'GRAY']
    깊은 우주에 어울리는 색상: ['BLACK', 'BLUE', 'PURPLE']
    에러 발생! ~~~: 잘못된 값입니다: '객체: 사과

    가능한 색상: 빨간색' (허용된 값: 빨간색, 초록색, 파란색, 갈색, 노란색, 검은색, 하얀색, 주황색, 보라색, 분홍색, 회색)
    For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
    12345에 어울리는 색상: ['RED', 'BLACK', 'GRAY']
    ```

* 모두 에러가 나올 것으로 예측했으나 그렇지 않았음
  * 색상이 출력된 내용: `내 마음속`, `푸른 바다`, `가방`
    * 사실상 의미 없는 단어들에 대해서도 모델이 가능한 색상 후보를 예측
    * 이는 언어 모델이 객체와 관련된 의미를 유추하거나, 기존 지식에서 색상과 유사한 연관을 찾아내기 때문
      * 예컨대 `푸른 바다` → `파란색`과 관련된 색상 후보가 나옴
      * `가방` → `흑색`, `갈색` 등을 추론해냄

  * 색상이 출력되지 않은 내용: `특수문자`, `숫자` = **에러**
    * 모델이 **정상적인 객체가 아니라는 것을 파악** → **입력에 맞는 색상 후보를 제시할 수 없음**

In [ ]:
# 에러 발생을 유도하기 위한 테스트 객체들
test_objects_with_errors = [
    "청록색",                     # 잘못된 색상 (Colors3에 없음)
    "빨강색",                     # 오타 (예: 빨간색 오타)
    " ",                        # 빈 문자열
    "파란색, 빨간색, 초록색, 보라색"   # 색상 개수 초과 (4개)
]

for obj in test_objects_with_errors:
    try:
        response = chain3.invoke({"object": obj})
        print(f"{obj}에 어울리는 색상:", [color.name for color in response])
    except OutputParserException as e:
        print(f"에러 발생! {obj}: {e}")
    except Exception as e:
        print(f"알 수 없는 에러 발생! {obj}: {e}")


<small>

* 셀 출력 (3.3s)
  
    ```markdown
    청록색에 어울리는 색상: ['RED', 'ORANGE', 'PURPLE']
    빨강색에 어울리는 색상: ['RED', 'ORANGE', 'YELLOW']
    에러 발생!  : 잘못된 값입니다: '죄송합니다. 객체에 대한 정보가 부족하여 어울리는 색상을 추천해 드릴 수 없습니다.

    객체가 무엇인지 알려주시면' (허용된 값: 빨간색, 초록색, 파란색, 갈색, 노란색, 검은색, 하얀색, 주황색, 보라색, 분홍색, 회색)
    For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
    파란색, 빨간색, 초록색, 보라색에 어울리는 색상: ['BLUE', 'RED', 'GREEN']
    ```

* 오타 처리 
  * `빨강색`, `청록색` → **유사도 체크** 를 통해 비슷한 색상 후보를 제시함
  * `빈 값` = 완전히 잘못된 값 입력 → **에외 발생**

---

* 결과
  * 모델이 특정 물체의 색상을 유추할 때, 의미가 없거나 전혀 관련 없는 단어에 대해서도 어떤 유추를 시도했기 때문에 예상보다 많은 색상들이 제시됨
  * 예외 처리가 필요한 입력에 대해서는 모델이 경고를 주고, 색상 목록을 정확하게 따르지 않으면 에러를 발생시키는 게 확인